In [1]:
import sys
import os
# setting path
sys.path.append('/work/scripts')
import pandas as pd
import seaborn as sns

In [2]:
import Sensitivity as sa
import output_utils as ou

# parameters influencing soil hydro/thermal state
### cmt_envground.txt
--------------------------------------------------------
#### tcsolid(m): Soil thermal conductivity for moss (W/mK)
current value: 0.25, range: 0.005-0.5

*Ekici et al. 2015,Jiang et al. 2015, O’Donnell et al. 2009*

#### tcsolid(f): Soil thermal conductivity for fibric (W/mK)
current value: 0.25, range: 0.005-0.5

*Ekici et al. 2015,Jiang et al. 2015, O’Donnell et al. 2009*

#### tcsolid(h): Soil thermal conductivity for humic (W/mK)
current value: 0.25, range: 0.02-2.0

*Ekici et al. 2015,Jiang et al. 2015, O’Donnell et al. 2009*

---------------------------------------------------------
#### porosity(m): porosity for moss layers (m3/m3)
current value: 0.98, range: 0.85-0.99

*O’Donnell et al. 2009*

#### porosity(f): porosity for fibric layers  (m3/m3)
current value: 0.95, range: 0.85-0.99

*O’Donnell et al. 2009*

#### porosity(h): porosity for humic layers  (m3/m3)
current value: 0.8, range: 0.7-0.9

*O’Donnell et al. 2009*

--------------------------------------------------------
#### bulkden(m): bulk density for moss (g/m3)
current value: 25,000, range: 10,000 - 80,000

*Tuomi et al. 2020, Rodionov et al. 2007, O’Donnell et al. 2009*

#### bulkden(f): bulk density for fibric (g/m3)
current value: 51,000, range: 20,000 - 200,000

*Tuomi et al. 2020, Rodionov et al. 2007, O’Donnell et al. 2009*

#### bulkden(h): bulk density for humic (g/m3)
current value: 176,000, range: 100,000 - 800,000

*Tuomi et al. 2020, Rodionov et al. 2007, O’Donnell et al. 2009*

--------------------------------------------------------
#### hksat(m): hydraulic conductivity at saturation for moss (mm/s)
current value: 0.15, range: 0.0002 - 30

*Ekici et al. 2015, Letts et al. 2000, Liu et al. 2019*

#### hksat(f): hydraulic conductivity at saturation for fibric (mm/s)
current value: 0.28, range: 0.0002 - 30

*Ekici et al. 2015, Letts et al. 2000, Liu et al. 2019*

#### hksat(h): hydraulic conductivity at saturation for humic (mm/s)
current value: 0.002, range: 0.00004 - 2.01

*Ekici et al. 2015, Letts et al. 2000, Liu et al. 2019*

---------------------------------------------------------
#### nfactor(s): Summer nfactor
current value: 1.5, range: 0.2-2.0

*Kade et al. 2006, Klene et al. 2001*

#### nfactor(w): Winter nfactor
current value: 1.0, range: 0.4-1.0

*Kade et al. 2006, Klene et al. 2001*

---------------------------------------------------------
#### snwalbmax
current value: 0.8, range: 0.7-0.85

*Te Beest et al. 2016, Petzold et al. 1975, Loranty et al. 2011*

#### snwalbmin
current value: 0.4, range: 0.4-0.6

*Te Beest et al. 2016, Petzold et al. 1975, Loranty et al. 2011*

---------------------------------------------------------
### cmt_dimground.txt
---------------------------------------------------------
#### snwdenmax (kg/m3)
current value: 250, range: 100-800

*Domine et al. 2016, Gerland et al. 1999, Muskett 2012*

#### snwdennew (kg/m3)
current value: 50, range: 10-250

*Domine et al. 2016, Gerland et al. 1999, Muskett 2012*


### cmt_bgcsoil.txt
---------------------------------------------------------
#### rhq10
current value: 2, range: 1.6-2.4
#### rhq10_w
current value: 2, range: 0.85-0.99

In [3]:


work_dir='/data/workflows/US-Prr_SWC_SA'
opt_run_setup='--tr-yrs=121 --sp-yrs=300 --eq-yrs=500 '

driver = sa.SensitivityDriver(work_dir = work_dir, clean=True)
driver.site = '/data/input-catalog/caribou-poker_merged/'
driver.opt_run_setup = opt_run_setup
driver.PXx ='1'
driver.PXy='0'

params = ['hksat(m)','hksat(f)','hksat(h)',
          'tcsolid(m)', 'tcsolid(f)', 'tcsolid(h)',
          'porosity(m)', 'porosity(f)', 'porosity(h)',
          'nfactor(s)', 'nfactor(w)',
          'rhq10']#, 'rhq10_w']
#hksat-m .12-.15
#.25-.35
#.01-0.33
bounds = [[0.0005056, 0.003922], [0.0000539, 0.004125], [0.000040, 0.000620], #[0.000040, ]
          [0.005, 0.5], [0.005, 0.5], [0.02, 2],
          [0.83, 0.98], [0.8, 0.83], [0.7, 0.8],
          [0.2, 2.5], [0.1, 1],
          [1.6, 2.4]]#, [1.6, 2.4]]


driver.logparams = [1, 1, 1,
                    1, 1, 1,
                    0, 0, 0,
                    0, 0,
                    0]#, 0]

driver.outputs = [
      { 'name': 'GPP', 'type': 'flux'},
      { 'name': 'RH','type': 'flux'},
      { 'name': 'ALD','type': 'flux'},
      { 'name': 'LWCLAYER','type': 'layer'},
      { 'name': 'VWCLAYER','type': 'layer'},
      { 'name': 'TLAYER','type': 'layer'},
      { 'name': 'LAYERDEPTH','type': 'layer'},
      { 'name': 'LAYERDZ','type': 'layer'},
      { 'name': 'LAYERTYPE','type': 'layer'},
    ]

driver.design_experiment(Nsamples = 50, cmtnum = 13, params = params, percent_diffs = percent_diffs,
                         bounds=bounds, pftnums = [None]*len(params), sampling_method='uniform')


In [4]:
driver.clean()
driver.setup_multi()


In [5]:
driver.sample_matrix

,hksat(m),hksat(f),hksat(h),tcsolid(m),tcsolid(f),tcsolid(h),porosity(m),porosity(f),porosity(h),nfactor(s),nfactor(w),rhq10
0,0.000715,0.000079,0.000172,0.008566,0.129852,0.027238,0.838713,0.825985,0.760112,1.828567,0.118526,2.375928
1,0.000895,0.000554,0.000046,0.378996,0.010094,0.632340,0.894792,0.808737,0.761185,0.520836,0.362930,1.893089
2,0.000727,0.000687,0.000101,0.090030,0.071047,0.052582,0.921132,0.805116,0.706505,2.382437,0.969069,2.246718
3,0.000606,0.001367,0.000058,0.023377,0.081734,1.250658,0.835158,0.827280,0.725878,1.723801,0.380540,2.016054
4,0.000647,0.000351,0.000048,0.009495,0.035256,0.051441,0.919685,0.827656,0.708849,0.650761,0.140705,1.860264
5,0.001299,0.000094,0.000603,0.193651,0.148545,0.048129,0.851139,0.824066,0.707455,2.469840,0.795020,1.758973
6,0.000772,0.000185,0.000097,0.086919,0.369576,0.023666,0.883770,0.803476,0.786310,1.633586,0.397808,1.650847
7,0.001066,0.000260,0.000368,0.058330,0.354901,0.175859,0.847939,0.821397,0.776079,1.490938,0.793870,1.995036
8,0.001418,0.000888,0.000080,0.306729,0.039870,0.269595,0.877153,0.815257,0.790757,0.773372,0.469345,2.204441
9,0.002080,0.000641,0.000259,0.188871,0.008423,0.027067,0.925011,0.826144,0.780367,0.629111,0.903303,2.031474


In [ ]:
driver.run_all_samples()

[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000008/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000000/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000006/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000004/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000002/config/config.js[SA:run] /work/dvmdostem --tr-yrs=121 --sp-yrs=300 --eq-yrs=500  -l err --force-cmt 13 --ctrl-file /data/workflows/US-Prr_SWC_SA/sample_000000010/config/config.js
[SA:run] /work/dvmdostem --